In [1]:
pip install python-docx pdfplumber groq

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.5/42.5 kB 1.8 MB/s eta 0:00:00
  Using cached typing_extensions-4.12.2-py3-none-any.whl.metadata (3.0 kB)
  Using cached pillow-11.1.0-cp312-cp312-manylinux_2_28_x86_64.whl.metadata (9.1 kB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.2/48.2 kB 4.1 MB/s eta 0:00:00
  Using cached charset_normalizer-3.4.1-cp312-cp312-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (35 kB)
  Using cached pydantic-2.10.6-py3-none-any.whl.metadata (30 kB)
  Using cached idna-3.10-py3-none-any.whl.metadata (10 kB)
  Using cached certifi-2025.1.31-py3-none-any.whl.metadata (2.5 kB)
  Using cached annotated_types-0.7.0-py3-none-any.whl.metadata (15 kB)
  Using cached pydantic_core-2.27.2-cp312-cp312-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (6.6 kB)
  Using cached cffi-1.17.1-cp312-cp312-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (1.5 kB)
  Using cached pycparser-2.22-py3-none-any.whl.metadata (943 bytes)
   ━━━━━━━━━━━━━

In [5]:
import os
from groq import Groq
from docx import Document
import pdfplumber

# Initialize Groq client with your API key
client = Groq(api_key="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")

# Function to extract text from DOCX
def extract_text_from_docx(file_path):
    doc = Document(file_path)
    full_text = []
    for para in doc.paragraphs:
        full_text.append(para.text)
    return "\n".join(full_text)

# Function to extract text from PDF
def extract_text_from_pdf(file_path):
    with pdfplumber.open(file_path) as pdf:
        full_text = ""
        for page in pdf.pages:
            full_text += page.extract_text()
    return full_text

# Function to extract information using Groq API
def extract_information_with_groq(text, template):
    # Prepare the prompt for Groq API
    prompt = f"""
    You are tasked with extracting financial and ESG-related information from the following text.
    Use the provided template to structure the output. Fill in the fields based on the content of the text.
    
    Template:
    {template}
    
    Text to analyze:
    {text}
    
    Output the filled template with the extracted information.
    """
    
    # Call Groq API
    completion = client.chat.completions.create(
        model="llama-3.3-70b-versatile",
        messages=[
            {"role": "system", "content": "You are an AI assistant that extracts structured information from unstructured text."},
            {"role": "user", "content": prompt}
        ],
        temperature=1,
        max_completion_tokens=1024,
        top_p=1,
        stream=True,
        stop=None,
    )
    
    # Stream the response
    extracted_info = ""
    for chunk in completion:
        extracted_info += chunk.choices[0].delta.content or ""
    
    return extracted_info

# Main function to process the file
def process_file(file_path, template):
    # Check file type and extract text
    if file_path.endswith('.docx'):
        text = extract_text_from_docx(file_path)
    elif file_path.endswith('.pdf'):
        text = extract_text_from_pdf(file_path)
    else:
        raise ValueError("Unsupported file format. Please upload a DOCX or PDF file.")
    
    # Extract information using Groq API
    extracted_data = extract_information_with_groq(text, template)
    
    return extracted_data

# Example usage
if __name__ == "__main__":
    # Define the comprehensive template
    template = """
    ### 1. Personal/Company Information
    - **Name**: [Insert Name]
    - **Organization**: [Insert Organization Name]
    - **Industry**: [Insert Industry Type]

    ### 2. Credit Information
    - **CIBIL Score**: [Insert CIBIL Score]
      - **Description**: [Brief description or notes about the score]

    ... (rest of the template)
    """
    
    # Path to the uploaded file (DOCX or PDF)
    file_path = "/home/dharun/Desktop/llama/temp.pdf"
    
    # Process the file and extract information
    try:
        extracted_info = process_file(file_path, template)
        print("Extracted Information:")
        print(extracted_info)
    except Exception as e:
        print(f"Error: {e}")

Extracted Information:
### 1. Personal/Company Information
- **Name**: Arjun Sharma
- **Organization**: Not explicitly stated (e-commerce business)
- **Industry**: E-commerce

### 2. Credit Information
- **CIBIL Score**: 785/900
  - **Description**: Demonstrates responsible financial behavior with timely repayments on his housing loan and no overdue or pending loans against his name.

### 3. Financial Information
- **Loan Amount**: ₹10,00,000
- **Annual Income**: ₹12,00,000
- **Business Turnover**: ₹25,00,000 annually
- **Monthly Expenses**: ₹60,000
- **Savings**: Fixed deposits worth ₹5,00,000, savings account balance averaging ₹2,00,000, and multiple bank accounts with combined balances totaling ₹3,50,000

### 4. Asset Information
- **Residential Property**: Valued at ₹80,00,000
- **Vehicles and Jewelry**: Worth ₹15,00,000
- **Agricultural Land**: Valued at ₹20,00,000 (owned by his family)

### 5. Employment and Business Experience
- **Years of Experience as an Entrepreneur**: 6 year

In [3]:
import os
from groq import Groq
from docx import Document
import pdfplumber

# Initialize Groq client with your API key
client = Groq(api_key="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")

# Define the list of valid keywords/phrases based on your categories
VALID_KEYWORDS = [
    "CIBIL Score", "Overdue", "Pending Loans", "Utility Payments", 
    "Electricity Bill", "Water Bill", "Digital Wallet", "Cost of Living", 
    "Annual Income", "Turnover", "Fixed Deposit", "Savings Investments", 
    "Assets", "Bank Balance", "Entrepreneur", "ESG", "Environmental Impact", 
    "Social Responsibility", "Governance"
]

# Function to extract text from DOCX
def extract_text_from_docx(file_path):
    doc = Document(file_path)
    full_text = []
    for para in doc.paragraphs:
        full_text.append(para.text)
    return "\n".join(full_text)

# Function to extract text from PDF
def extract_text_from_pdf(file_path):
    with pdfplumber.open(file_path) as pdf:
        full_text = ""
        for page in pdf.pages:
            full_text += page.extract_text()
    return full_text

# Function to validate content based on keywords
def validate_content(text):
    # Check if any of the valid keywords are present in the text
    for keyword in VALID_KEYWORDS:
        if keyword.lower() in text.lower():
            return True
    return False

# Function to extract information using Groq API
def extract_information_with_groq(text, template):
    # Prepare the prompt for Groq API
    prompt = f"""
    You are tasked with extracting financial and ESG-related information from the following text.
    Use the provided template to structure the output. Fill in the fields based on the content of the text.
    
    Template:
    {template}
    
    Text to analyze:
    {text}
    
    Output the filled template with the extracted information.
    """
    
    # Call Groq API
    completion = client.chat.completions.create(
        model="llama-3.3-70b-versatile",
        messages=[
            {"role": "system", "content": "You are an AI assistant that extracts structured information from unstructured text."},
            {"role": "user", "content": prompt}
        ],
        temperature=1,
        max_completion_tokens=1024,
        top_p=1,
        stream=True,
        stop=None,
    )
    
    # Stream the response
    extracted_info = ""
    for chunk in completion:
        extracted_info += chunk.choices[0].delta.content or ""
    
    return extracted_info

# Main function to process the file
def process_file(file_path, template):
    # Check file type and extract text
    if file_path.endswith('.docx'):
        text = extract_text_from_docx(file_path)
    elif file_path.endswith('.pdf'):
        text = extract_text_from_pdf(file_path)
    else:
        raise ValueError("Unsupported file format. Please upload a DOCX or PDF file.")
    
    # Validate the content before proceeding
    if not validate_content(text):
        raise ValueError("The uploaded file does not contain relevant financial or ESG-related information.")
    
    # Extract information using Groq API
    extracted_data = extract_information_with_groq(text, template)
    
    return extracted_data

# Example usage
if __name__ == "__main__":
    # Define the comprehensive template
    template = """
    ### 1. Personal/Company Information
    - **Name**: [Insert Name]
    - **Organization**: [Insert Organization Name]
    - **Industry**: [Insert Industry Type]

    ### 2. Credit Information
    - **CIBIL Score**: [Insert CIBIL Score]
      - **Description**: [Brief description or notes about the score]

    ... (rest of the template)
    """
    
    # Path to the uploaded file (DOCX or PDF)
    file_path = "/home/dharun/Desktop/llama/A Personalized Approach to Post-Traumatic Stress Disorder 1 (1) (1).pdf"
    
    # Process the file and extract information
    try:
        extracted_info = process_file(file_path, template)
        print("Extracted Information:")
        print(extracted_info)
    except Exception as e:
        print(f"Error: {e}")

Error: The uploaded file does not contain relevant financial or ESG-related information.


In [2]:
import os
from groq import Groq
from docx import Document
import pdfplumber

# Initialize Groq client with your API key
client = Groq(api_key="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")

# Define the list of valid keywords/phrases based on your categories
VALID_KEYWORDS = [
    "CIBIL Score", "Overdue", "Pending Loans", "Utility Payments", 
    "Electricity Bill", "Water Bill", "Digital Wallet", "Cost of Living", 
    "Annual Income", "Turnover", "Fixed Deposit", "Savings Investments", 
    "Assets", "Bank Balance", "Entrepreneur", "ESG", "Environmental Impact", 
    "Social Responsibility", "Governance"
]

# Define thresholds for each condition
CONDITION_THRESHOLDS = {
    "CIBIL Score": {"max_score": 2, "good_threshold": 750},
    "Overdue/Pending Loans": {"max_score": 2, "max_overdue": 50000},
    "Utility Payments": {"max_score": 1, "on_time_ratio": 0.9},
    "Digital Wallet Transactions": {"max_score": 1, "min_transactions": 5},
    "Cost of Living vs Spending": {"max_score": 1, "savings_ratio": 0.2},
    "Annual Income or Turnover": {"max_score": 2, "min_income": 500000},
    "Fixed Deposit/Savings Investments": {"max_score": 1, "min_investment": 100000},
    "Assets of Individual/Family": {"max_score": 1, "min_assets": 500000},
    "Bank Balance & Multiple Accounts": {"max_score": 1, "min_balance": 100000},
    "Entrepreneur Length of Employment": {"max_score": 1, "min_years": 2},
    "ESG Related Information": {"max_score": 1, "min_initiatives": 1}
}

# Function to extract text from DOCX
def extract_text_from_docx(file_path):
    doc = Document(file_path)
    full_text = []
    for para in doc.paragraphs:
        full_text.append(para.text)
    return "\n".join(full_text)

# Function to extract text from PDF
def extract_text_from_pdf(file_path):
    with pdfplumber.open(file_path) as pdf:
        full_text = ""
        for page in pdf.pages:
            full_text += page.extract_text()
    return full_text

# Function to validate content based on keywords
def validate_content(text):
    # Check if any of the valid keywords are present in the text
    for keyword in VALID_KEYWORDS:
        if keyword.lower() in text.lower():
            return True
    return False

# Function to extract information using Groq API
def extract_information_with_groq(text, template):
    # Prepare the prompt for Groq API
    prompt = f"""
    You are tasked with extracting financial and ESG-related information from the following text.
    Use the provided template to structure the output. Fill in the fields based on the content of the text.
    
    Template:
    {template}
    
    Text to analyze:
    {text}
    
    Output the filled template with the extracted information.
    """
    
    # Call Groq API
    completion = client.chat.completions.create(
        model="llama-3.3-70b-versatile",
        messages=[
            {"role": "system", "content": "You are an AI assistant that extracts structured information from unstructured text."},
            {"role": "user", "content": prompt}
        ],
        temperature=1,
        max_completion_tokens=1024,
        top_p=1,
        stream=True,
        stop=None,
    )
    
    # Stream the response and parse it into a dictionary
    extracted_info = {}
    current_key = None
    for chunk in completion:
        content = chunk.choices[0].delta.content or ""
        lines = content.split("\n")
        for line in lines:
            if line.strip().endswith(":"):
                # This is a key (e.g., "CIBIL Score:")
                current_key = line.strip().rstrip(":")
                extracted_info[current_key] = ""
            elif current_key:
                # This is a value for the current key
                extracted_info[current_key] += line.strip()
    
    return extracted_info

# Function to calculate score for each condition
def calculate_score(extracted_data):
    total_score = 0
    
    # Example: CIBIL Score
    cibil_score = int(extracted_data.get("CIBIL Score", 0))
    if cibil_score >= CONDITION_THRESHOLDS["CIBIL Score"]["good_threshold"]:
        total_score += CONDITION_THRESHOLDS["CIBIL Score"]["max_score"]
    
    # Example: Overdue/Pending Loans
    overdue_loans = int(extracted_data.get("Overdue/Pending Loans", 0))
    if overdue_loans <= CONDITION_THRESHOLDS["Overdue/Pending Loans"]["max_overdue"]:
        total_score += CONDITION_THRESHOLDS["Overdue/Pending Loans"]["max_score"]
    
    # Example: Utility Payments
    utility_payment_ratio = float(extracted_data.get("Utility Payments On-Time Ratio", 0))
    if utility_payment_ratio >= CONDITION_THRESHOLDS["Utility Payments"]["on_time_ratio"]:
        total_score += CONDITION_THRESHOLDS["Utility Payments"]["max_score"]
    
    # Example: Digital Wallet Transactions
    digital_wallet_transactions = int(extracted_data.get("Digital Wallet Transactions", 0))
    if digital_wallet_transactions >= CONDITION_THRESHOLDS["Digital Wallet Transactions"]["min_transactions"]:
        total_score += CONDITION_THRESHOLDS["Digital Wallet Transactions"]["max_score"]
    
    # Example: Cost of Living vs Spending
    savings_ratio = float(extracted_data.get("Savings Ratio", 0))
    if savings_ratio >= CONDITION_THRESHOLDS["Cost of Living vs Spending"]["savings_ratio"]:
        total_score += CONDITION_THRESHOLDS["Cost of Living vs Spending"]["max_score"]
    
    # Example: Annual Income or Turnover
    annual_income = int(extracted_data.get("Annual Income", 0))
    if annual_income >= CONDITION_THRESHOLDS["Annual Income or Turnover"]["min_income"]:
        total_score += CONDITION_THRESHOLDS["Annual Income or Turnover"]["max_score"]
    
    # Example: Fixed Deposit/Savings Investments
    savings_investments = int(extracted_data.get("Fixed Deposit/Savings Investments", 0))
    if savings_investments >= CONDITION_THRESHOLDS["Fixed Deposit/Savings Investments"]["min_investment"]:
        total_score += CONDITION_THRESHOLDS["Fixed Deposit/Savings Investments"]["max_score"]
    
    # Example: Assets of Individual/Family
    assets = int(extracted_data.get("Assets", 0))
    if assets >= CONDITION_THRESHOLDS["Assets of Individual/Family"]["min_assets"]:
        total_score += CONDITION_THRESHOLDS["Assets of Individual/Family"]["max_score"]
    
    # Example: Bank Balance & Multiple Accounts
    bank_balance = int(extracted_data.get("Bank Balance", 0))
    if bank_balance >= CONDITION_THRESHOLDS["Bank Balance & Multiple Accounts"]["min_balance"]:
        total_score += CONDITION_THRESHOLDS["Bank Balance & Multiple Accounts"]["max_score"]
    
    # Example: Entrepreneur Length of Employment
    employment_years = int(extracted_data.get("Entrepreneur Length of Employment", 0))
    if employment_years >= CONDITION_THRESHOLDS["Entrepreneur Length of Employment"]["min_years"]:
        total_score += CONDITION_THRESHOLDS["Entrepreneur Length of Employment"]["max_score"]
    
    # Example: ESG Related Information
    esg_initiatives = int(extracted_data.get("ESG Initiatives", 0))
    if esg_initiatives >= CONDITION_THRESHOLDS["ESG Related Information"]["min_initiatives"]:
        total_score += CONDITION_THRESHOLDS["ESG Related Information"]["max_score"]
    
    return total_score

# Main function to process the file
def process_file(file_path, template):
    # Check file type and extract text
    if file_path.endswith('.docx'):
        text = extract_text_from_docx(file_path)
    elif file_path.endswith('.pdf'):
        text = extract_text_from_pdf(file_path)
    else:
        raise ValueError("Unsupported file format. Please upload a DOCX or PDF file.")
    
    # Validate the content before proceeding
    if not validate_content(text):
        raise ValueError("The uploaded file does not contain relevant financial or ESG-related information.")
    
    # Extract information using Groq API
    extracted_data = extract_information_with_groq(text, template)
    
    # Calculate total score
    total_score = calculate_score(extracted_data)
    
    # Determine loan eligibility
    if total_score >= 8:
        return f"Loan Approved! Total Score: {total_score}/10"
    else:
        return f"Loan Rejected. Total Score: {total_score}/10"

# Example usage
if __name__ == "__main__":
    # Define the comprehensive template
    template = """
    ### 1. Personal/Company Information
    - **Name**: [Insert Name]
    - **Organization**: [Insert Organization Name]
    - **Industry**: [Insert Industry Type]

    ### 2. Credit Information
    - **CIBIL Score**: [Insert CIBIL Score]
      - **Description**: [Brief description or notes about the score]

    ... (rest of the template)
    """
    
    # Path to the uploaded file (DOCX or PDF)
    file_path = "/home/dharun/Desktop/llama/A Personalized Approach to Post-Traumatic Stress Disorder 1 (1) (1).pdf"
    
    # Process the file and determine loan eligibility
    try:
        result = process_file(file_path, template)
        print(result)
    except Exception as e:
        print(f"Error: {e}")

Error: The uploaded file does not contain relevant financial or ESG-related information.


In [1]:
import os
from groq import Groq
from docx import Document
import pdfplumber

# Initialize Groq client with your API key
client = Groq(api_key="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")

# Define the list of valid keywords/phrases based on your categories
VALID_KEYWORDS = [
    "CIBIL Score", "Overdue", "Pending Loans", "Utility Payments", 
    "Electricity Bill", "Water Bill", "Digital Wallet", "Cost of Living", 
    "Annual Income", "Turnover", "Fixed Deposit", "Savings Investments", 
    "Assets", "Bank Balance", "Entrepreneur", "ESG", "Environmental Impact", 
    "Social Responsibility", "Governance"
]

# Define thresholds for each condition
CONDITION_THRESHOLDS = {
    "CIBIL Score": {"max_score": 2, "good_threshold": 750},
    "Overdue/Pending Loans": {"max_score": 2, "max_overdue": 50000},
    "Utility Payments": {"max_score": 1, "on_time_ratio": 0.9},
    "Digital Wallet Transactions": {"max_score": 1, "min_transactions": 5},
    "Cost of Living vs Spending": {"max_score": 1, "savings_ratio": 0.2},
    "Annual Income or Turnover": {"max_score": 2, "min_income": 500000},
    "Fixed Deposit/Savings Investments": {"max_score": 1, "min_investment": 100000},
    "Assets of Individual/Family": {"max_score": 1, "min_assets": 500000},
    "Bank Balance & Multiple Accounts": {"max_score": 1, "min_balance": 100000},
    "Entrepreneur Length of Employment": {"max_score": 1, "min_years": 2},
    "ESG Related Information": {"max_score": 1, "min_initiatives": 1}
}

# Function to extract text from DOCX
def extract_text_from_docx(file_path):
    doc = Document(file_path)
    full_text = []
    for para in doc.paragraphs:
        full_text.append(para.text)
    return "\n".join(full_text)

# Function to extract text from PDF
def extract_text_from_pdf(file_path):
    with pdfplumber.open(file_path) as pdf:
        full_text = ""
        for page in pdf.pages:
            full_text += page.extract_text()
    return full_text

# Function to validate content based on keywords
def validate_content(text):
    # Check if any of the valid keywords are present in the text
    for keyword in VALID_KEYWORDS:
        if keyword.lower() in text.lower():
            return True
    return False

# Function to extract information using Groq API
def extract_information_with_groq(text, template):
    # Prepare the prompt for Groq API
    prompt = f"""
    You are tasked with extracting financial and ESG-related information from the following text.
    Use the provided template to structure the output. Fill in the fields based on the content of the text.
    
    Template:
    {template}
    
    Text to analyze:
    {text}
    
    Output the filled template with the extracted information.
    """
    
    # Call Groq API
    completion = client.chat.completions.create(
        model="llama-3.3-70b-versatile",
        messages=[
            {"role": "system", "content": "You are an AI assistant that extracts structured information from unstructured text."},
            {"role": "user", "content": prompt}
        ],
        temperature=1,
        max_completion_tokens=1024,
        top_p=1,
        stream=True,
        stop=None,
    )
    
    # Stream the response and parse it into a dictionary
    extracted_info = {}
    current_key = None
    for chunk in completion:
        content = chunk.choices[0].delta.content or ""
        lines = content.split("\n")
        for line in lines:
            if line.strip().endswith(":"):
                # This is a key (e.g., "CIBIL Score:")
                current_key = line.strip().rstrip(":")
                extracted_info[current_key] = ""
            elif current_key:
                # This is a value for the current key
                extracted_info[current_key] += line.strip()
    
    return extracted_info

# Function to calculate score and adjust threshold dynamically
def calculate_score_and_adjust_threshold(extracted_data, original_threshold=8):
    total_score = 0
    max_possible_score = 0
    missing_conditions = []

    # Example: CIBIL Score
    cibil_score = int(extracted_data.get("CIBIL Score", 0))
    if cibil_score > 0:
        max_possible_score += CONDITION_THRESHOLDS["CIBIL Score"]["max_score"]
        if cibil_score >= CONDITION_THRESHOLDS["CIBIL Score"]["good_threshold"]:
            total_score += CONDITION_THRESHOLDS["CIBIL Score"]["max_score"]
    else:
        missing_conditions.append("CIBIL Score")

    # Example: Overdue/Pending Loans
    overdue_loans = int(extracted_data.get("Overdue/Pending Loans", 0))
    if overdue_loans > 0:
        max_possible_score += CONDITION_THRESHOLDS["Overdue/Pending Loans"]["max_score"]
        if overdue_loans <= CONDITION_THRESHOLDS["Overdue/Pending Loans"]["max_overdue"]:
            total_score += CONDITION_THRESHOLDS["Overdue/Pending Loans"]["max_score"]
    else:
        missing_conditions.append("Overdue/Pending Loans")

    # Repeat similar logic for other conditions...

    # Adjust the threshold based on missing conditions
    num_conditions = len(CONDITION_THRESHOLDS)
    num_missing = len(missing_conditions)
    adjusted_threshold = original_threshold * (num_conditions - num_missing) / num_conditions

    return total_score, adjusted_threshold, missing_conditions


# Main function to process the file
def process_file(file_path, template):
    # Check file type and extract text
    if file_path.endswith('.docx'):
        text = extract_text_from_docx(file_path)
    elif file_path.endswith('.pdf'):
        text = extract_text_from_pdf(file_path)
    else:
        raise ValueError("Unsupported file format. Please upload a DOCX or PDF file.")
    
    # Validate the content before proceeding
    if not validate_content(text):
        raise ValueError("The uploaded file does not contain relevant financial or ESG-related information.")
    
    # Extract information using Groq API
    extracted_data = extract_information_with_groq(text, template)
    
    # Calculate total score and adjust threshold
    total_score, adjusted_threshold, missing_conditions = calculate_score_and_adjust_threshold(extracted_data)
    
    # Determine loan eligibility based on 75% of the original threshold
    original_threshold = 8
    minimum_required_threshold = original_threshold * 0.75
    
    if total_score >= minimum_required_threshold:
        return "You are eligible for a loan! Our bank will reach you shortly."
    else:
        rejection_reason = (
            f"Loan Rejected. Total Score: {total_score}/{adjusted_threshold:.1f}\n"
            f"The following conditions were missing: {', '.join(missing_conditions)}.\n"
            f"Your score did not meet the minimum required threshold of {minimum_required_threshold:.1f}."
        )
        return rejection_reason


# Example usage
if __name__ == "__main__":
    # Define the comprehensive template
    template = """
    ### 1. Personal/Company Information
    - **Name**: [Insert Name]
    - **Organization**: [Insert Organization Name]
    - **Industry**: [Insert Industry Type]

    ### 2. Credit Information
    - **CIBIL Score**: [Insert CIBIL Score]
      - **Description**: [Brief description or notes about the score]

    ... (rest of the template)
    """
    
    # Path to the uploaded file (DOCX or PDF)
    file_path = "/home/dharun/Desktop/llama/A Personalized Approach to Post-Traumatic Stress Disorder 1 (1) (1).pdf"
    
    # Process the file and determine loan eligibility
    try:
        result = process_file(file_path, template)
        print(result)
    except Exception as e:
        print(f"Error: {e}")

Error: The uploaded file does not contain relevant financial or ESG-related information.


In [5]:
import os
from groq import Groq
from docx import Document
import pdfplumber

# Initialize Groq client with your API key
client = Groq(api_key="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")

# Define thresholds for each condition
CONDITION_THRESHOLDS = {
    "CIBIL Score": {"max_score": 2, "good_threshold": 750},
    "Overdue/Pending Loans": {"max_score": 2, "max_overdue": 50000},
    "Utility Payments": {"max_score": 1, "on_time_ratio": 0.9},
    "Digital Wallet Transactions": {"max_score": 1, "min_transactions": 5},
    "Cost of Living vs Spending": {"max_score": 1, "savings_ratio": 0.2},
    "Annual Income or Turnover": {"max_score": 2, "min_income": 500000},
    "Fixed Deposit/Savings Investments": {"max_score": 1, "min_investment": 100000},
    "Assets of Individual/Family": {"max_score": 1, "min_assets": 500000},
    "Bank Balance & Multiple Accounts": {"max_score": 1, "min_balance": 100000},
    "Entrepreneur Length of Employment": {"max_score": 1, "min_years": 2},
    "ESG Related Information": {"max_score": 1, "min_initiatives": 1}
}

# Function to extract text from DOCX
def extract_text_from_docx(file_path):
    doc = Document(file_path)
    full_text = []
    for para in doc.paragraphs:
        full_text.append(para.text)
    return "\n".join(full_text)

# Function to extract text from PDF
def extract_text_from_pdf(file_path):
    with pdfplumber.open(file_path) as pdf:
        full_text = ""
        for page in pdf.pages:
            full_text += page.extract_text()
    return full_text

# Function to extract information using Groq API
def extract_information_with_groq(text):
    # Prepare the prompt for Groq API
    prompt = f"""
    You are tasked with extracting financial and ESG-related information from the following text.
    Identify and extract the following fields if they are present in the text:
    - CIBIL Score
    - Overdue/Pending Loans
    - Utility Payments (On-Time Ratio)
    - Digital Wallet Transactions
    - Cost of Living vs Spending (Savings Ratio)
    - Annual Income or Turnover
    - Fixed Deposit/Savings Investments
    - Assets of Individual/Family
    - Bank Balance & Multiple Accounts
    - Entrepreneur Length of Employment
    - ESG Related Information (Number of Initiatives)
    
    Text to analyze:
    {text}
    
    Output the extracted information in the format: Field: Value
    """
    
    # Call Groq API
    completion = client.chat.completions.create(
        model="llama-3.3-70b-versatile",
        messages=[
            {"role": "system", "content": "You are an AI assistant that extracts structured information from unstructured text."},
            {"role": "user", "content": prompt}
        ],
        temperature=1,
        max_completion_tokens=1024,
        top_p=1,
        stream=True,
        stop=None,
    )
    
    # Stream the response and parse it into a dictionary
    extracted_info = {}
    current_key = None
    for chunk in completion:
        content = chunk.choices[0].delta.content or ""
        lines = content.split("\n")
        for line in lines:
            if ":" in line:
                key, value = line.split(":", 1)
                extracted_info[key.strip()] = value.strip()
    
    return extracted_info

# Function to calculate score and adjust threshold dynamically
def calculate_score_and_adjust_threshold(extracted_data, original_threshold=8):
    total_score = 0
    max_possible_score = 0
    missing_conditions = []

    # Example: CIBIL Score
    cibil_score = int(extracted_data.get("CIBIL Score", 0))
    if cibil_score > 0:
        max_possible_score += CONDITION_THRESHOLDS["CIBIL Score"]["max_score"]
        if cibil_score >= CONDITION_THRESHOLDS["CIBIL Score"]["good_threshold"]:
            total_score += CONDITION_THRESHOLDS["CIBIL Score"]["max_score"]
    else:
        missing_conditions.append("CIBIL Score")

    # Example: Overdue/Pending Loans
    overdue_loans = int(extracted_data.get("Overdue/Pending Loans", 0))
    if overdue_loans > 0:
        max_possible_score += CONDITION_THRESHOLDS["Overdue/Pending Loans"]["max_score"]
        if overdue_loans <= CONDITION_THRESHOLDS["Overdue/Pending Loans"]["max_overdue"]:
            total_score += CONDITION_THRESHOLDS["Overdue/Pending Loans"]["max_score"]
    else:
        missing_conditions.append("Overdue/Pending Loans")

    # Repeat similar logic for other conditions...

    # Adjust the threshold based on missing conditions
    num_conditions = len(CONDITION_THRESHOLDS)
    num_missing = len(missing_conditions)
    adjusted_threshold = original_threshold * (num_conditions - num_missing) / num_conditions

    return total_score, adjusted_threshold, missing_conditions


# Main function to process the file
def process_file(file_path):
    # Check file type and extract text
    if file_path.endswith('.docx'):
        text = extract_text_from_docx(file_path)
    elif file_path.endswith('.pdf'):
        text = extract_text_from_pdf(file_path)
    else:
        raise ValueError("Unsupported file format. Please upload a DOCX or PDF file.")
    
    # Extract information using Groq API
    extracted_data = extract_information_with_groq(text)
    
    # Calculate total score and adjust threshold
    total_score, adjusted_threshold, missing_conditions = calculate_score_and_adjust_threshold(extracted_data)
    
    # Determine loan eligibility based on 75% of the original threshold
    original_threshold = 8
    minimum_required_threshold = original_threshold * 0.75
    
    if total_score >= minimum_required_threshold:
        return "You are eligible for a loan! Our bank will reach you shortly."
    else:
        rejection_reason = (
            f"Loan Rejected. Total Score: {total_score}/{adjusted_threshold:.1f}\n"
            f"The following conditions were missing: {', '.join(missing_conditions)}.\n"
            f"Your score did not meet the minimum required threshold of {minimum_required_threshold:.1f}."
        )
        return rejection_reason


# Example usage
if __name__ == "__main__":
    # Path to the uploaded file (DOCX or PDF)
    file_path = "/home/dharun/Desktop/llama/A Personalized Approach to Post-Traumatic Stress Disorder 1 (1) (1).pdf"
    
    # Process the file and determine loan eligibility
    try:
        result = process_file(file_path)
        print(result)
    except Exception as e:
        print(f"Error: {e}")

Error: Error code: 413 - {'error': {'message': 'Request too large for model `llama-3.3-70b-versatile` in organization `org_01jmaafhskfcf8tc0wcg2fcaz1` service tier `on_demand` on tokens per minute (TPM): Limit 6000, Requested 13127, please reduce your message size and try again. Visit https://console.groq.com/docs/rate-limits for more information.', 'type': 'tokens', 'code': 'rate_limit_exceeded'}}


In [8]:
import os
from groq import Groq
from docx import Document
import pdfplumber

# Initialize Groq client with your API key
client = Groq(api_key="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")

# Define thresholds for each condition
CONDITION_THRESHOLDS = {
    "CIBIL Score": {"max_score": 2, "good_threshold": 750},
    "Overdue/Pending Loans": {"max_score": 2, "max_overdue": 50000},
    "Utility Payments": {"max_score": 1, "on_time_ratio": 0.9},
    "Digital Wallet Transactions": {"max_score": 1, "min_transactions": 5},
    "Cost of Living vs Spending": {"max_score": 1, "savings_ratio": 0.2},
    "Annual Income or Turnover": {"max_score": 2, "min_income": 500000},
    "Fixed Deposit/Savings Investments": {"max_score": 1, "min_investment": 100000},
    "Assets of Individual/Family": {"max_score": 1, "min_assets": 500000},
    "Bank Balance & Multiple Accounts": {"max_score": 1, "min_balance": 100000},
    "Entrepreneur Length of Employment": {"max_score": 1, "min_years": 2},
    "ESG Related Information": {"max_score": 1, "min_initiatives": 1}
}

# Function to extract text from DOCX
def extract_text_from_docx(file_path):
    doc = Document(file_path)
    full_text = []
    for para in doc.paragraphs:
        full_text.append(para.text)
    return "\n".join(full_text)

# Function to extract text from PDF
def extract_text_from_pdf(file_path):
    with pdfplumber.open(file_path) as pdf:
        full_text = ""
        for page in pdf.pages:
            full_text += page.extract_text()
    return full_text

# Function to extract information using Groq API
def extract_information_with_groq(text):
    # Prepare the prompt for Groq API
    prompt = f"""
    You are tasked with extracting financial and ESG-related information from the following text.
    Identify and extract the following fields if they are present in the text:
    - CIBIL Score
    - Overdue/Pending Loans
    - Utility Payments (On-Time Ratio)
    - Digital Wallet Transactions
    - Cost of Living vs Spending (Savings Ratio)
    - Annual Income or Turnover
    - Fixed Deposit/Savings Investments
    - Assets of Individual/Family
    - Bank Balance & Multiple Accounts
    - Entrepreneur Length of Employment
    - ESG Related Information (Number of Initiatives)
    
    Text to analyze:
    {text}
    
    Output the extracted information in the format: Field: Value
    """
    
    # Call Groq API with updated parameters
    completion = client.chat.completions.create(
        model="qwen-2.5-32b",  # Updated model name
        messages=[
            {"role": "system", "content": "You are an AI assistant that extracts structured information from unstructured text."},
            {"role": "user", "content": prompt}
        ],
        temperature=0.6,  # Adjusted temperature
        max_completion_tokens=4096,  # Increased max tokens
        top_p=0.95,  # Adjusted top_p
        stream=True,
        stop=None,
    )
    
    # Stream the response and parse it into a dictionary
    extracted_info = {}
    current_key = None
    for chunk in completion:
        content = chunk.choices[0].delta.content or ""
        lines = content.split("\n")
        for line in lines:
            if ":" in line:
                key, value = line.split(":", 1)
                extracted_info[key.strip()] = value.strip()
    
    return extracted_info

# Function to calculate score and adjust threshold dynamically
def calculate_score_and_adjust_threshold(extracted_data, original_threshold=8):
    total_score = 0
    max_possible_score = 0
    missing_conditions = []

    # Example: CIBIL Score
    cibil_score = int(extracted_data.get("CIBIL Score", 0))
    if cibil_score > 0:
        max_possible_score += CONDITION_THRESHOLDS["CIBIL Score"]["max_score"]
        if cibil_score >= CONDITION_THRESHOLDS["CIBIL Score"]["good_threshold"]:
            total_score += CONDITION_THRESHOLDS["CIBIL Score"]["max_score"]
    else:
        missing_conditions.append("CIBIL Score")

    # Example: Overdue/Pending Loans
    overdue_loans = int(extracted_data.get("Overdue/Pending Loans", 0))
    if overdue_loans > 0:
        max_possible_score += CONDITION_THRESHOLDS["Overdue/Pending Loans"]["max_score"]
        if overdue_loans <= CONDITION_THRESHOLDS["Overdue/Pending Loans"]["max_overdue"]:
            total_score += CONDITION_THRESHOLDS["Overdue/Pending Loans"]["max_score"]
    else:
        missing_conditions.append("Overdue/Pending Loans")

    # Repeat similar logic for other conditions...

    # Adjust the threshold based on missing conditions
    num_conditions = len(CONDITION_THRESHOLDS)
    num_missing = len(missing_conditions)
    adjusted_threshold = original_threshold * (num_conditions - num_missing) / num_conditions

    return total_score, adjusted_threshold, missing_conditions


# Main function to process the file
def process_file(file_path):
    # Check file type and extract text
    if file_path.endswith('.docx'):
        text = extract_text_from_docx(file_path)
    elif file_path.endswith('.pdf'):
        text = extract_text_from_pdf(file_path)
    else:
        raise ValueError("Unsupported file format. Please upload a DOCX or PDF file.")
    
    # Extract information using Groq API
    extracted_data = extract_information_with_groq(text)
    
    # Calculate total score and adjust threshold
    total_score, adjusted_threshold, missing_conditions = calculate_score_and_adjust_threshold(extracted_data)
    
    # Determine loan eligibility based on 75% of the original threshold
    original_threshold = 8
    minimum_required_threshold = original_threshold * 0.75
    
    if total_score >= minimum_required_threshold:
        return "You are eligible for a loan! Our bank will reach you shortly."
    else:
        rejection_reason = (
            f"Loan Rejected. Total Score: {total_score}/{adjusted_threshold:.1f}\n"
            f"The following conditions were missing: {', '.join(missing_conditions)}.\n"
            f"Your score did not meet the minimum required threshold of {minimum_required_threshold:.1f}."
        )
        return rejection_reason


# Example usage
if __name__ == "__main__":
    # Path to the uploaded file (DOCX or PDF)
    file_path = "/home/dharun/Desktop/llama/temp.pdf"
    
    # Process the file and determine loan eligibility
    try:
        result = process_file(file_path)
        print(result)
    except Exception as e:
        print(f"Error: {e}")

Loan Rejected. Total Score: 0/6.5
The following conditions were missing: CIBIL Score, Overdue/Pending Loans.
Your score did not meet the minimum required threshold of 6.0.


In [ ]:
import os
from groq import Groq
from docx import Document
import pdfplumber

# Initialize Groq client with your API key
client = Groq(api_key="")

# Define simplified thresholds for each condition
CONDITION_THRESHOLDS = {
    "CIBIL Score": {"max_score": 1, "good_threshold": 600},  # Relaxed threshold
    "Annual Income or Turnover": {"max_score": 1, "min_income": 300000},  # Lower income requirement
    "Bank Balance & Multiple Accounts": {"max_score": 1, "min_balance": 50000},  # Lower balance requirement
    "Utility Payments": {"max_score": 1, "on_time_ratio": 0.7},  # Relaxed payment history
    "ESG Related Information": {"max_score": 1, "min_initiatives": 0}  # Optional ESG initiatives
}

# Function to extract text from DOCX
def extract_text_from_docx(file_path):
    doc = Document(file_path)
    full_text = []
    for para in doc.paragraphs:
        full_text.append(para.text)
    return "\n".join(full_text)

# Function to extract text from PDF
def extract_text_from_pdf(file_path):
    with pdfplumber.open(file_path) as pdf:
        full_text = ""
        for page in pdf.pages:
            full_text += page.extract_text()
    return full_text

# Function to extract information using Groq API
def extract_information_with_groq(text):
    # Prepare the prompt for Groq API
    prompt = f"""
    You are tasked with extracting financial and ESG-related information from the following text.
    Identify and extract the following fields if they are present in the text:
    - CIBIL Score
    - Annual Income or Turnover
    - Bank Balance & Multiple Accounts
    - Utility Payments (On-Time Ratio)
    - ESG Related Information (Number of Initiatives)
    
    Text to analyze:
    {text}
    
    Output the extracted information in the format: Field: Value
    """
    
    # Call Groq API with updated parameters
    completion = client.chat.completions.create(
        model="qwen-2.5-32b",  # Updated model name
        messages=[
            {"role": "system", "content": "You are an AI assistant that extracts structured information from unstructured text."},
            {"role": "user", "content": prompt}
        ],
        temperature=0.6,  # Adjusted temperature
        max_completion_tokens=4096,  # Increased max tokens
        top_p=0.95,  # Adjusted top_p
        stream=True,
        stop=None,
    )
    
    # Stream the response and parse it into a dictionary
    extracted_info = {}
    current_key = None
    for chunk in completion:
        content = chunk.choices[0].delta.content or ""
        lines = content.split("\n")
        for line in lines:
            if ":" in line:
                key, value = line.split(":", 1)
                extracted_info[key.strip()] = value.strip()
    
    return extracted_info

# Function to calculate score and adjust threshold dynamically
def calculate_score_and_adjust_threshold(extracted_data, original_threshold=5):
    total_score = 0
    missing_conditions = []

    # Example: CIBIL Score
    cibil_score = int(extracted_data.get("CIBIL Score", 0))
    if cibil_score > 0:
        if cibil_score >= CONDITION_THRESHOLDS["CIBIL Score"]["good_threshold"]:
            total_score += CONDITION_THRESHOLDS["CIBIL Score"]["max_score"]
    else:
        missing_conditions.append("CIBIL Score")

    # Example: Annual Income or Turnover
    annual_income = int(extracted_data.get("Annual Income or Turnover", 0))
    if annual_income > 0:
        if annual_income >= CONDITION_THRESHOLDS["Annual Income or Turnover"]["min_income"]:
            total_score += CONDITION_THRESHOLDS["Annual Income or Turnover"]["max_score"]
    else:
        missing_conditions.append("Annual Income or Turnover")

    # Example: Bank Balance & Multiple Accounts
    bank_balance = int(extracted_data.get("Bank Balance & Multiple Accounts", 0))
    if bank_balance > 0:
        if bank_balance >= CONDITION_THRESHOLDS["Bank Balance & Multiple Accounts"]["min_balance"]:
            total_score += CONDITION_THRESHOLDS["Bank Balance & Multiple Accounts"]["max_score"]
    else:
        missing_conditions.append("Bank Balance & Multiple Accounts")

    # Example: Utility Payments
    utility_payment_ratio = float(extracted_data.get("Utility Payments", 0))
    if utility_payment_ratio > 0:
        if utility_payment_ratio >= CONDITION_THRESHOLDS["Utility Payments"]["on_time_ratio"]:
            total_score += CONDITION_THRESHOLDS["Utility Payments"]["max_score"]
    else:
        missing_conditions.append("Utility Payments")

    # Example: ESG Related Information
    esg_initiatives = int(extracted_data.get("ESG Related Information", 0))
    if esg_initiatives >= CONDITION_THRESHOLDS["ESG Related Information"]["min_initiatives"]:
        total_score += CONDITION_THRESHOLDS["ESG Related Information"]["max_score"]

    # Adjust the threshold based on missing conditions
    num_conditions = len(CONDITION_THRESHOLDS)
    num_missing = len(missing_conditions)
    adjusted_threshold = original_threshold * (num_conditions - num_missing) / num_conditions

    return total_score, adjusted_threshold, missing_conditions


# Main function to process the file
def process_file(file_path):
    # Check file type and extract text
    if file_path.endswith('.docx'):
        text = extract_text_from_docx(file_path)
    elif file_path.endswith('.pdf'):
        text = extract_text_from_pdf(file_path)
    else:
        raise ValueError("Unsupported file format. Please upload a DOCX or PDF file.")
    
    # Extract information using Groq API
    extracted_data = extract_information_with_groq(text)
    
    # Calculate total score and adjust threshold
    total_score, adjusted_threshold, missing_conditions = calculate_score_and_adjust_threshold(extracted_data)
    
    # Determine loan eligibility based on relaxed threshold
    original_threshold = 5  # Reduced threshold
    minimum_required_threshold = original_threshold * 0.75
    
    if total_score >= minimum_required_threshold:
        return "Great news! You are eligible for a loan. Our team will contact you soon."
    else:
        rejection_reason = (
            f"We're sorry, but your application does not meet the requirements.\n"
            f"Total Score: {total_score}/{adjusted_threshold:.1f}\n"
            f"The following conditions were missing: {', '.join(missing_conditions)}.\n"
            f"To qualify, aim to improve these areas."
        )
        return rejection_reason


# Example usage
if __name__ == "__main__":
    # Path to the uploaded file (DOCX or PDF)
    file_path = "/home/dharun/Desktop/llama/temp.pdf"
    
    # Process the file and determine loan eligibility
    try:
        result = process_file(file_path)
        print(result)
    except Exception as e:
        print(f"Error: {e}")

We're sorry, but your application does not meet the requirements.
Total Score: 1/1.0
The following conditions were missing: CIBIL Score, Annual Income or Turnover, Bank Balance & Multiple Accounts, Utility Payments.
To qualify, aim to improve these areas.
